# Node2Vec

In [2]:
!pip install numpy
!pip install networkx
!pip install gensim
!pip install pandas
!pip install matplotlib
!pip install scikit-learn

In [17]:
import pandas as pd
import numpy as np
import networkx as nx
from gensim.models import Word2Vec

from multiprocessing import cpu_count
import random

from gensim.models.keyedvectors import KeyedVectors

from collections import defaultdict
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle as skshuffle

import networkx as nx
import random
from gensim.models import Word2Vec # Needed for Word2Vec

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import Reddit
from torch_geometric.datasets import HeterophilousGraphDataset

from torch_geometric.utils import to_networkx

### DATA AND HYPERPARAMETERS

In [18]:
DATA_FOLDER = '/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/Datasets/'
NAME_DATASET = 'BlogCatalog-dataset'
edges_path = DATA_FOLDER + NAME_DATASET + '/data/edges.csv'
nodes_path = DATA_FOLDER + NAME_DATASET + '/data/nodes.csv'
groups_path = DATA_FOLDER + NAME_DATASET + '/data/groups.csv'
group_edges_path = DATA_FOLDER + NAME_DATASET + '/data/group-edges.csv'

In [19]:
nodes_id = pd.read_csv(nodes_path, header=None, names=['id'])
groups_id = pd.read_csv(groups_path, header=None, names=['group'])
edges = pd.read_csv(edges_path, header=None, names=['id_1', 'id_2'])
user_group_membership = pd.read_csv(group_edges_path, header=None, names=['id', 'group'])

In [20]:
# Create a graph
G = nx.Graph()

# Add nodes to the graph
G.add_nodes_from(nodes_id['id'])

# Add edges to the graph
G.add_edges_from(edges[['id_1', 'id_2']].values)

#Add weight
for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = 1

alias_nodes = {}

is_directed = False

In [21]:
# Create a dictionary to store groups for each ID
group_dict = {}

# Populate the group_dict
for _, row in user_group_membership.iterrows():
    user_id = row['id']
    group_id = row['group']

    # Check if the user_id is already in the dictionary
    if user_id in group_dict:
        group_dict[user_id].append(group_id)
    else:
        group_dict[user_id] = [group_id]

# Add group labels to the nodes
for user_id, groups in group_dict.items():
    nx.set_node_attributes(G, {user_id: groups}, 'group_belonging')

# Print basic graph information
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 10312
Number of edges: 333983


In [22]:
def get_set_labels(G):
    '''
    Input:
        G: Graph
    Output:
        labels: List of sets containing labels. Each set corresponds to the groups of id = index + 1
    '''

    labels = []

    for n in G.nodes:
        l = set(G.nodes[n].get('group_belonging', []))  # Ensure an empty list is converted to an empty set
        labels.append(l)

    return labels

def get_array_labels(G):
  '''
  Input:
    G: Graph
  Output:
    labels: Array with labels. Each position of the array will correspond to the groups of id = index + 1
  '''

  labels = []

  for n in G.nodes:
      l = G.nodes[n].get('group_belonging')
      labels.append(l)
  return labels

In [23]:
# HYPERPARAMETERS
window_size = 10
embedding_size = 128
walks_per_vertex = 10 ##10 --> Parameters suggested in Khosla 
walk_length = 80 ##80 --> Parameters suggested in Khosla

p = 0.25 # Possibilities evaluated in Khosla: 0.25, 0.5, 1, 2, 4
q = 4 # Possibilities evaluated in Khosla: 0.25, 0.5, 1, 2, 4

### NODE2VEC IMPLEMENTATION

In [24]:
# Effective Discrete Sampling. Idea from: https://lips.cs.princeton.edu/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
def alias_setup(probs):
    '''
    Input:
        probs: array containing the probabilities of different outvomes
    Output:
        J: indices that represent the outcome
        q: probabilities after being sampled
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int64)

    smaller = [] # Probs smaller than 1/K
    larger = [] # Probs bigger than 1/K

    for kk, prob in enumerate(probs):
        q[kk] = K*prob # Scaling the probability
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0

        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

In [25]:
def alias_draw(J, q):
    # Generates random samples from the given distribution.
    '''
    Inputs:
        J: indices that represent the outcome
        q: probabilities after being sampled
    '''
    K = len(J)

    # Draw from the overall uniform mixture. --> Integer in the range [0, K]
    kk = int(np.floor(np.random.rand() * K))

    # Draw from the binary mixture, either keeping the small one, or choosing the associated larger one.
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [26]:
# Method to compute the transition probabilities, according to section 3.2.2 of the Node2Vec paper
def get_alias_edge(G, src, dst):
    '''
    Inputs:
        G: Graph
        src: Node where the edge starts, source
        dst: Node where the edge finishes, destination
    '''
    unnormalized_probs = [] # We will set the probs according to section 3.2.2
    for dst_nghbr in sorted(G.neighbors(dst)):
        if dst_nghbr == src:
            unnormalized_probs.append(G[dst][dst_nghbr]['weight']/p) # distance 0 --> prob 1/p (for not weighted)
        elif G.has_edge(dst_nghbr, src):
            unnormalized_probs.append(G[dst][dst_nghbr]['weight']) # distance 1 --> prob 1 (for not weighted)
        else:
            unnormalized_probs.append(G[dst][dst_nghbr]['weight']/q) # distance 2 --> prob 1/q (for not weighted)

    normalizing_factor = sum(unnormalized_probs)
    normalized_probs = [float(un_p)/normalizing_factor for un_p in unnormalized_probs]

    return alias_setup(normalized_probs)

In [27]:
def preprocess_transition_probs(G, is_directed):
    '''
    Inputs:
        G: graph
        is_directed: indicates if the graph is direced or undirected
    Ouputs:
        alias_nodes: Dictionary where the keys are nodes in the graph, and the values are the corresponding alias tables for each node. 
        alias_edges: Dictionary where the keys are edges in the graph, and the values are the corresponding alias tables for each edge.
    '''

    alias_nodes = {}
    for node in G.nodes():
        unnormalized_probs = [G[node][nghbr]['weight'] for nghbr in sorted(G.neighbors(node))] # Get the transition probabilities from a node to its neighbours
        norm_const = sum(unnormalized_probs) # Compute the normalization constant
        normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs] # Compute the normalized probs
        alias_nodes[node] = alias_setup(normalized_probs) # Compute the alias table for the node, and store it in the dictionary.

    alias_edges = {}

    print("First part finished")

    if is_directed:
        for edge in G.edges():
            alias_edges[edge] = get_alias_edge(G, edge[0], edge[1]) # Compute the alias table for the given edges
    else: # In undirected graphs, the edge (node_1, node_2) is the same as the (node_2, node_1)
        for edge in G.edges():
            alias_edges[edge] = get_alias_edge(G, edge[0], edge[1])
            alias_edges[(edge[1], edge[0])] = get_alias_edge(G, edge[1], edge[0]) # Compute and store the alias table for the reverse direction.

    return alias_nodes, alias_edges

In [28]:
def node2vec_walk(G, alias_nodes, alias_edges, walk_length, start_node):
    '''
    Compute the randdom walk given the first node
    Input:
        G: graph
        alias_nodes: alias tables for nodes
        alias_edges: alias tables for edges
        walk_length: number of steps in the walk
        start_node: first node of the walk
    Output:
        walk: a list containing the nodes visited
    '''

    walk = [start_node]

    while len(walk) < walk_length:
        current_node = walk[-1] # Get the last node visited
        current_nbrs = sorted(G.neighbors(current_node)) # Get the neighbours of the current node

        if len(current_nbrs) > 0: # Check if the node has neighbours
            if len(walk) == 1: # First node of the walk
                walk.append(current_nbrs[alias_draw(alias_nodes[current_node][0], alias_nodes[current_node][1])]) # Get the next node to visit
            else:
                prev = walk[-2]
                next = current_nbrs[alias_draw(alias_edges[(prev, current_node)][0], alias_edges[(prev, current_node)][1])] # Sample the next node
                walk.append(next)
        else: # If the current node has no neighbours, the walk is over
            break

    return walk

In [29]:
def simulate_walks(G, alias_nodes, alias_edges, num_walks, walk_length):
    '''
    Generate all the necessary walks
    
    Inputs:
        G: graph
        alias_nodes: alias tables for nodes
        alias_edges: alias tables for edges
        num_walks: number of walks that start from the same vertex
        walk_length: number of steps in the walk
    '''
    random.seed(1)
    np.random.seed(1)
    walks = []
    nodes = list(G.nodes())
    print('Walk iteration:')
    for walk_iter in range(num_walks):
        print(str(walk_iter+1), '/', str(num_walks))
        random.shuffle(nodes)
        for node in nodes:
            walks.append(node2vec_walk(G, alias_nodes, alias_edges, walk_length=walk_length, start_node=node))

    return walks

In [30]:
# Optimization
def learn_embeddings(walks):
    walks = [list(map(str, walk)) for walk in walks]
    model = Word2Vec(walks,
                     vector_size=embedding_size,
                     window=window_size,
                     min_count=0,
                     sg=1,
                     workers=cpu_count(),
                     epochs=1) # Number of iterations in the SGD. In the paper they do 1.

    return model

In [31]:
alias_nodes, alias_edges = preprocess_transition_probs(G, is_directed)
walks = simulate_walks(G, alias_nodes, alias_edges, walks_per_vertex, walk_length)
phi = learn_embeddings(walks)

First part finished
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [32]:
# Get the embedding vectors
phi_vectors = phi.wv.vectors

In [33]:
# SAVE the model in Word2Vec format
phi.wv.save_word2vec_format('/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/embeddings_node2vec/blogcatalog.embedding')

In [85]:
# LOAD the model in Word2Vec format
phi = KeyedVectors.load_word2vec_format('/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/embeddings_node2vec/blogcatalog.embedding')

In [41]:
# Check the LOADED embedding vectors
phi_vectors = phi.vectors
print(phi_vectors)

[[ 0.12204421 -0.11647057  0.26194656 ... -0.00428664 -0.16073275
  -0.055148  ]
 [ 0.07751068  0.0205436   0.05751998 ... -0.07587191 -0.12947647
  -0.09578805]
 [ 0.11857373 -0.0989447   0.15763068 ...  0.02885029  0.27141353
   0.09615699]
 ...
 [-0.04881622 -0.0916038   0.11737502 ... -0.25610986  0.01920853
   0.24216694]
 [ 0.28862426 -0.25290966  0.20937659 ... -0.04427637  0.16115342
  -0.05660533]
 [ 0.25468144 -0.34269187  0.00873971 ... -0.16506337 -0.02325846
   0.20982495]]


### NODE CLASSIFIER

In [36]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [37]:
# To improve the access to the adjacency matrix, we pass the info to a dictionary, where all the nodes have a entry, and the values aare the neighbours
def adj_dictionary(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo() # Return a COOrdinate representation of this matrix
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k, v in G.items()}

ATTENTION!
In the following method, some lines need to be commented depending on:
- The type of dataset that is being used
- The index of the first node
- If the embedding has been loaded or generated

In [43]:
labels_blog = get_array_labels(G)

adj_G = nx.adjacency_matrix(G, nodelist=None, dtype=None, weight='weight') # Adjacency matrix: Square matrix of N x N size used to represent the connections between the edges of a graph.
adj_graph = adj_dictionary(adj_G)

num_labels = len(groups_id)
#num_labels = dataset.num_classes ### FOR PYTORCH_GEOMETRIC DATASETS
mlb = MultiLabelBinarizer(classes=range(1, len(groups_id) + 1))
#mlb = MultiLabelBinarizer(classes=range(num_labels))  ### FOR PYTORCH_GEOMETRIC DATASETS THAT START WITH 0

labels_bin = mlb.fit_transform(labels_blog)

### WHEN USING A LOADED EMBEDDING, ERASE THE .wv
try:
    features_matrix = np.asarray([phi.wv[str(node+1)] for node in range(len(G))])
except:
    features_matrix = np.asarray([phi.wv[str(node)] for node in range(len(G))])

shuffles = []
for x in range(10):
    shuffles.append(skshuffle(features_matrix, labels_bin)) # The sklearn shuffle shuffles arrays or sparse matrices in a consistent way.

all_results = defaultdict(list)

training_percents = [0.1, 0.5, 0.9]

for train_percent in training_percents:
    for shuf in shuffles:
        X, y = shuf # X corresponds to the phi_vectors and y to the labels.

        training_size = int(train_percent * len(X))

        X_train = X[:training_size, :]
        y_train_ = y[:training_size]

        y_train = [[] for x in range(len(y_train_))]

        y_train_sparse = coo_matrix(y_train_)
        cy =  y_train_sparse.tocoo()

        for i, j in zip(cy.row, cy.col):
            y_train[i].append(j)

        assert sum(len(l) for l in y_train) == np.count_nonzero(y_train_)

        X_test = X[training_size:, :]
        y_test_ = y[training_size:]

        y_test = [[] for _ in range(len(y_test_))]

        y_test_sparse = coo_matrix(y_test_)
        cy =  y_test_sparse.tocoo()
        for i, j in zip(cy.row, cy.col):
            y_test[i].append(j)

        clf = TopKRanker(LogisticRegression()) # creates an instance of TopKRanker with a LogisticRegression model as the base classifier.
        clf.fit(X_train, y_train_)

        # find out how many labels should be predicted
        top_k_list = [len(l) for l in y_test]
        preds = clf.predict(X_test, top_k_list)

        results = {}
        averages = ["micro", "macro"]
        for average in averages:
            results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)

        all_results[train_percent].append(results)


/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/Node2vec/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [0] will be ignored
  warnings.warn(
/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/Node2vec/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [0] will be ignored
  warnings.warn(
/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/Node2vec/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/Nod

In [44]:
print ('Results, using embeddings of dimensionality', X.shape[1])
print ('-------------------')
for train_percent in sorted(all_results.keys()):
    print ('Train percent:', train_percent)
for index, result in enumerate(all_results[train_percent]):
    print ('Shuffle #%d:   ' % (index + 1), result)
avg_score = defaultdict(float)
for score_dict in all_results[train_percent]:
    for metric, score in score_dict.items():
        avg_score[metric] += score
for metric in avg_score:
    avg_score[metric] /= len(all_results[train_percent])
print ('Average score:', dict(avg_score))
print ('-------------------')

Results, using embeddings of dimensionality 128
-------------------
Train percent: 0.1
Train percent: 0.5
Train percent: 0.9
Shuffle #1:    {'micro': 0.39767680218653906, 'macro': 0.2435402820456096}
Shuffle #2:    {'micro': 0.3778990450204638, 'macro': 0.2297999862887019}
Shuffle #3:    {'micro': 0.4099895941727368, 'macro': 0.26055351855327674}
Shuffle #4:    {'micro': 0.40161453077699294, 'macro': 0.2756815577505518}
Shuffle #5:    {'micro': 0.4063380281690141, 'macro': 0.27646647911798367}
Shuffle #6:    {'micro': 0.40168835736897646, 'macro': 0.2530218639191589}
Shuffle #7:    {'micro': 0.4089793055068397, 'macro': 0.28383539755641424}
Shuffle #8:    {'micro': 0.39667705088265837, 'macro': 0.24010178394756224}
Shuffle #9:    {'micro': 0.40618411806043564, 'macro': 0.25485157220420146}
Shuffle #10:    {'micro': 0.391169368747844, 'macro': 0.2518558357289587}
Average score: {'micro': 0.3998216200892501, 'macro': 0.2569708277112419}
-------------------
